##Warning! Almost nothing is in functions

In [5]:
#provide directory in which BASS is located
bass_folder_loc = '/Users/morganfine-morris/Documents/Repos/bass'
#uncomment '%matplotlib inline' below see plots inline (and remove blocking). 
#If you change your mind after running it, you must restart kernel
#%matplotlib inline 
#%matplotlib qt

import sys
import os
import os.path as path
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
# append the path to the folder containing bass 
# (which should prob. contain a __init__.py file.) to pythonpath
sys.path.append(bass_folder_loc) 
import bass
reload(bass)

BASS ready!


<module 'bass' from '/Users/morganfine-morris/Documents/Repos/bass/bass.pyc'>

In [2]:
#initialize new file
Data = {}
Settings = {}
Results ={}

############################################################################################
#manual Setting block
Settings['folder']= r"./"#Physionet_ECGs"
Settings['Label'] = 'p01_r01_f.txt'#r'sel_100_tabsep.csv'
Settings['Output Folder'] = 'output'

#transformation Settings
Settings['Absolute Value'] = True #Must be True if Savitzky-Golay is being used

Settings['Bandpass Highcut'] =  'none'#in Hz
Settings['Bandpass Lowcut'] = 'none' #in Hz
Settings['Bandpass Polynomial'] = 'none' #integer

Settings['Linear Fit'] = False #between 0 and 1 on the whole time series
Settings['Linear Fit-Rolling R'] = 0.75 #between 0 and 1
Settings['Linear Fit-Rolling Window'] = 1000 #window for rolling mean for fit, unit is index not time
Settings['Relative Baseline'] = 0 #default 0, unless data is normalized, then 1.0. Can be any float

#integer
Settings['Savitzky-Golay Polynomial'] = 4 
#must be odd. units are index not time
Settings['Savitzky-Golay Window Size'] = 'none'#27 

#Baseline Settings
Settings['Baseline Type'] = r'static' #'linear', 'rolling', or 'static'
#For Linear
Settings['Baseline Start'] = 10.04 #start time in seconds
Settings['Baseline Stop'] = 10.18 #end time in seconds
#For Rolling
Settings['Rolling Baseline Window'] = 1 # in seconds. leave as 'none' if linear or static

#Peaks
Settings['Delta'] = .2#0.25
Settings['Peak Minimum'] = -1 #amplitude value
Settings['Peak Maximum'] = 1 #amplitude value

#Bursts
Settings['Burst Area'] = False #calculate burst area
Settings['Exclude Edges'] = False #False to keep edges, True to discard them
Settings['Inter-event interval minimum (seconds)'] = 0.00100 #only for bursts, not for peaks
Settings['Maximum Burst Duration (s)'] = 10 
Settings['Minimum Burst Duration (s)'] = 0
Settings['Minimum Peak Number'] = 1 #minimum number of peaks/burst, integer
#Settings['Threshold']= 4.9#.3 #linear: proportion of baseline. 
                           #static: literal value.
                           #rolling, linear ammount grater than rolling baseline at each time point.
if Settings['Baseline Type']=='rolling':
    Settings['Threshold']= .01 #linear: proportion of baseline. 
elif Settings['Baseline Type']=='static':
    Settings['Threshold']= 0.01#4.9 #linear: proportion of baseline. 
else:
    pass
#Outputs
Settings['Generate Graphs'] = False #create and save the fancy graph outputs


#Settings that you should not change unless you are a super advanced user:
#These are settings that are still in development
Settings['Graph LCpro events'] = False
Settings['File Type'] = r'Plain' #'LCPro', 'ImageJ', 'SIMA', 'Plain', 'Morgan'
Settings['Milliseconds'] = False
############################################################################################

In [225]:
#remove header from file and resave file (overwrites original data)
path_file = path.join(Settings['folder'], Settings['Label'])
file_data = pd.DataFrame.from_csv(path_file, header=0, sep="\t")

try:
    new = pd.DataFrame(file_data.values, file_data.index.values, dtype=float)
except:
    #its a string, remove it
    new = pd.DataFrame(file_data.values[1:], file_data.index.values[1:])

new.to_csv(path_file, sep='\t',header=False)

#Data Processing (not file specific)

In [3]:
reload(bass)
Data, Settings = bass.load_wrapper({},Settings)
#Data, Settings = bass.transform_wrapper(Data, Settings) #b/c I can't find smoothing settings that don't make it worse
Data['trans'] = Data['original']
Data, Settings, Results = bass.baseline_wrapper(Data, Settings, Results)
#event detection
Results = bass.event_peakdet_wrapper(Data, Settings, Results)
Results = bass.event_burstdet_wrapper(Data, Settings, Results)

#clip the data from s seconds to e seconds
s=1
e=50
Data_clipped = {'original':Data['original'][s:e], 'trans':Data['trans'][s:e]}
#print Data_clipped
# graph_ts calls show, so unless you ran %matplotlib inline, 
# notebook will block until you close the plot window
#Data['original']['Mean2'][s:e].plot()
#plt.show()
axes = bass.graph_ts(Data_clipped, Settings, Results, roi="Mean1") 
plt.show()

BASS ready!
Made plots folder
Data Loaded
Sampling Rate= 0.002 (sec/frame)
p01_r01_f.txt is 19.992 seconds long.


In [192]:
#spectrogram
version = 'original'
key = 'Mean1'
bass.spectogram(version, key, Data, Settings, Results)
plt.show()

In [9]:
for key in ['Mean1']:
    #represent what a complete feature looks like for this particular ECG
    # s = burst start, e = burst end, v = valley, p=peak
    features_key = 'vspevspe'
    
    peak_count = features_key.count('p')
    valley_count = features_key.count('v')

    peaks_df = Results['Peaks'][key].dropna() # R wave location
    valleys_df = Results['Valleys'][key].dropna() # Q and S wave locations
    bursts_df = None
    
    try:
        burst_ends = Results['Bursts'][key]['Burst End']
        burst_starts = Results['Bursts'][key]['Burst Start']
        
        bursts_df = pd.concat([burst_starts, burst_ends], axis=1)
        bursts_df.columns = ['Start','End']
        #print bursts_df
    except:
        print "No bursts."
        
    #get peak and valley times
    print (peaks_df.head())
    peak_time = peaks_df.index.values
    valley_time = valleys_df.index.values
    
print peaks_df["Peaks Amplitude"].head()
n, pEdges = np.histogram(peaks_df["Peaks Amplitude"],bins=peak_count)
n, vEdges = np.histogram(valleys_df,bins=valley_count)
print pEdges, vEdges

       Peaks Amplitude  Intervals
Time                             
0.704            0.625      0.234
0.938            0.175      0.516
1.454            0.595      0.220
1.674            0.175      0.594
2.268            0.735      0.222
Time
0.704    0.625
0.938    0.175
1.454    0.595
1.674    0.175
2.268    0.735
Name: Peaks Amplitude, dtype: float64
0.175    4
0.620    4
0.210    3
0.550    3
0.165    3
0.650    2
0.195    2
0.610    2
0.575    1
0.615    1
0.690    1
0.150    1
0.285    1
0.595    1
0.495    1
0.180    1
0.490    1
0.600    1
0.160    1
0.670    1
0.245    1
0.655    1
0.215    1
0.200    1
0.795    1
0.185    1
0.480    1
0.225    1
0.325    1
0.735    1
0.190    1
0.625    1
dtype: int64
[ 0.15    0.4725  0.795 ] [-0.235 -0.135 -0.035]


#Getting Q, R, S, and T in the DataFrame and Ordered according to Time

Once you can see that Q, R, S, and T are being consistently detected (T should be marked by the burst ends) you can do the below process to label them. 

In [193]:
peaks_df = Results['Peaks']['Mean1'] # R wave location
valleys_df = Results['Valleys']['Mean1'] # Q and S wave locations
burst_ends = Results['Bursts']['Mean1']['Burst End'] #burst ends indicate approx. T wave location

# make new DataFrame for Burst Ends which is indexed by Burst Ends 
# b.c. Results['Bursts']['Mean1'] is indexed by Burst Start.
bursts_df = pd.DataFrame(burst_ends.values, index=burst_ends.values, columns=['Burst Ends'])

#combine peaks and valleys dfs and sort by time
# this will allow us to easily find peaks, 
# and find valleys before and after.
peaks_df['time'] = peaks_df.index
valleys_df['time'] = valleys_df.index
bursts_df['time'] = bursts_df.index
peaks_df['amplitude'] = peaks_df["Peaks Amplitude"]
valleys_df['amplitude'] = valleys_df["Valley Amplitude"]
bursts_df['amplitude'] = float(Settings['Threshold'])

#Label R and T. Unfortunatly, setting Q and S (the valleys) is much harder (See next cell).
peaks_df['letter'] = 'R'
bursts_df['letter'] = "T"

combined = peaks_df.append(valleys_df).sort('time')

#now combine w. Burst Ends and remove unnecessary columns, sort by time again
times = combined.append(bursts_df).drop([u'Burst Ends', u'Intervals', u'Peaks Amplitude', u'Valley Amplitude'],axis=1)
sorted_times = times.sort('time')
#remove na vals only from amplitude col.
sorted_times['amplitude'] = sorted_times['amplitude']
sorted_times.head()

KeyError: 'Mean1'

The following is a hack-y way to set the Q and S valleys, but it works. Later I might try to do it with booleans. <b>Feel free to try another way to do this!!! (Actually, really, please do. This method is pretty slow.)</b>

In [165]:
#reindex by integer
sorted_times['num'] = range(len(sorted_times))
sorted_times_reindexed = sorted_times.set_index('num')

letters = pd.DataFrame(index=sorted_times_reindexed.index, columns=['letters'], dtype=str)
for i in sorted_times_reindexed.index:
    
    #try to get the previous and next values with error handeling for edge cases
    try:
        prev_value = sorted_times_reindexed.loc[i-1, "letter"]
    except:
        prev_value=''
    try:
        next_value = sorted_times_reindexed.loc[i+1,"letter"]
    except:
        next_value=""
    
    #letters['time'][i] = sorted_times_reindexed.loc[i,'time']
    if (next_value == "T") and (prev_value == "R"):
        letters['letters'][i] = "S"
    elif next_value == "R":
        letters['letters'][i] = "Q"
    else:
        letters['letters'][i] = sorted_times_reindexed.loc[i,'letter']
        
sorted_times_reindexed['letter'] = letters['letters']
sorted_times_reindexed.head(13) #see that valleys which are not immediately before or after R are left as NaN!

,amplitude,letter,time
num,,,
0,-0.487,Q,0.12950
1,1.410,R,0.13475
2,-0.883,S,0.13825
3,0.030,T,0.15575
4,-0.455,Q,0.30125
5,1.514,R,0.30700
6,-0.934,S,0.31050
7,0.030,T,0.32575
8,-0.376,Q,0.47350


In [147]:
sorted_times_time_indexed = sorted_times_reindexed.set_index('time')
final = sorted_times_time_indexed.dropna()
final.head()

,amplitude,letter
time,,
0.12950,-0.487,Q
0.13475,1.410,R
0.13825,-0.883,S
0.15575,0.030,T
0.30125,-0.455,Q


In [153]:
s1=0
e1=20
final[s1:e1].plot()
plt.show()